## C01 Analyzing Performance
### Timing Julia Functions
- `@time` measure the time of expression run; **not return the time they used**
- `@timev` enhanced `time`; **not return the time they used**
- `@elapsed` **return the time of expression run**

<mark style="background: #DB4D6D;">**Note**: for the first execute expression, the timing will record the time used for compile</mark>


In [1]:
@time sqrt.(rand(1000));

  0.000015 seconds (2 allocations: 15.875 KiB)


In [2]:
@time for i in 1:1000
    x = sin.(rand(1000))
end

  0.012573 seconds (2.00 k allocations: 15.503 MiB)


In [3]:
@timev sqrt.(rand(1000));

  0.000010 seconds (2 allocations: 15.875 KiB)
elapsed time (ns)

:  10387
gc time (ns):       0
bytes allocated:    16256
pool allocs:        0
non-pool GC allocs: 2
minor collections:  0
full collections:   0


In [4]:
a = @time sqrt.(rand(1000));
sum(@time sqrt.(rand(1000)));
println("\na = " * string(a))

  0.000012 seconds (2 allocations: 15.875 KiB)
  0.000011 seconds (2 allocations: 15.875 KiB)



a = [0.5721389865673566, 0.4095122784381285, 0.5951351884540552, 0.9782295221654991, 0.6391840220709095, 0.9185261635173686, 0.994341870743468, 0.6499100387462687, 0.9854435053656161, 0.7041976914343921, 0.598913882336577, 0.6376316321057297, 0.8985321783386664, 0.1374636866399775, 0.07216340196107471, 0.8967786223415228, 0.6955965155343523, 0.342760200719822, 0.8582729359308607, 0.7146077233505418, 0.5200815007786008, 0.4634134367296263, 0.4634136534617932, 0.8790062813431863, 0.8471862579005801, 0.6609554891266604, 0.8369505787828216, 0.6184007575730726, 0.637934843045257, 0.427228089681921, 0.5859848606488868, 0.5403380795759188, 0.8859602754503687, 0.8218710235273895, 0.6138274749935112, 0.913827905396697, 0.3842245694692903, 0.61021114454824, 0.7311960149034075, 0.8289801507996251, 0.9454889111347502, 0.8121082716838366, 0.7824050184297408, 0.8995718588250087, 0.5214260819049206, 0.8855292885336278, 0.5444276233026477, 0.4554823787157653, 0.9364935301913514, 0.2975104044928526, 0

In [5]:
a = @elapsed sqrt.(rand(1000))
println("a = " * string(a))

a = 9.261e-6


In [6]:
using Test
@test @elapsed(sqrt.(rand(1000))) <= 1e-3

Test Passed

### The Julia Profiler
> The profiler is used to identify bottlenecks in code

- `Profile`
  - `Profile.clear()`: clear the profile info stored in memory.
  - `@profile`: record the profile of expression
  - `Profile.print()`: show the profile info of expression.
  - `Profile.init(delay=::Number)`: setting the *sampling interval*. defualt is `1ms` (`delay=.001`) in Linux. `.01` is `10ms`
- The result of Profile
  - The first field is the count time, which is determined by the interval of count. The larger number of count times represents this part take longer time to run.
  - For long-running program, we need to set larger interval.
- `TimerOutputs`: **for long-runing program**
  - `const to = TimerOutput();`: define a global `TimerOutput`, and pass it to `to`
  - `@time to::TimerOutput "name"::String`
  - `print_timer(to::TimerOutput)`

In [7]:
using Profile

In [8]:
using Statistics
function randmsq()
    x = rand(10000,1000)
    y = mean(x .^2, dims=1)
    return y
end

randmsq (generic function with 1 method)

In [9]:
randmsq();
Profile.clear()
Profile.init(delay = .001)
@profile randmsq();
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎16  @Base/Base.jl:608; (::Base.var"#1055#1056")()
   ╎ 16  @Base/Base.jl:572; profile_printing_listener()
   ╎  16  @Base/asyncevent.jl:159; wait
   ╎   16  @Base/asyncevent.jl:142; _trywait(t::Base.AsyncCondition)
   ╎    16  @Base/condition.jl:125; wait
   ╎     16  @Base/condition.jl:130; wait(c::Base.GenericCondition...
   ╎    ╎ 16  @Base/task.jl:994; wait()
 16╎    ╎  16  @Base/task.jl:985; poptask(W::Base.IntrusiveLin...
   ╎16  @Base/client.jl:552; _start()
   ╎ 16  @Base/client.jl:318; exec_options(opts::Base.JLOptions)
   ╎  16  @Base/Base.jl:495; include(mod::Module, _path::Str...
   ╎   16  @Base/loading.jl:2130; _include(mapexpr::Function, mo...
   ╎    16  @Base/loading.jl:2070; include_string(mapexpr::typeo...
   ╎     16  @Base/boot.jl:385; eval
   ╎    ╎ 16  ...tebook/notebook.jl:32; top-level scope
   ╎    ╎  16  .../serve_notebook.jl:75; kwcall(::@NamedTuple{crashre...
   ╎    ╎   16  ...serve_notebook.jl:

In [10]:
using TimerOutputs
const to = TimerOutput();

function randmsq_timed()
    @timeit to "randmsq" begin
        x = @timeit to "rand" rand(10000,1000)
        y = @timeit to "mean" mean(x .^2, dims=1)
    end
end

randmsq_timed (generic function with 1 method)

In [11]:
randmsq_timed();
print_timer(to)

 ────────────────────────────────────────────────────────────────────
                            Time                    Allocations      
                   ───────────────────────   ────────────────────────
 Tot / % measured:      330ms /  14.6%            174MiB /  87.8%    

 Section   ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────
 randmsq        1   48.3ms  100.0%  48.3ms    153MiB  100.0%   153MiB
   rand         1   26.4ms   54.7%  26.4ms   76.3MiB   50.0%  76.3MiB
   mean         1   21.8ms   45.2%  21.8ms   76.3MiB   50.0%  76.3MiB
 ────────────────────────────────────────────────────────────────────


In [12]:
const to_myself = TimerOutput();
function testmy()
    @timeit to_myself "forloop" for i in 1:1000
            x = sin.(rand(1000))
    end
    @timeit to_myself "parallel_for" begin
        Threads.@threads for i in 1:1000
            x = sin.(rand(1000))
        end
    end
end

testmy (generic function with 1 method)

In [13]:
testmy();
print_timer(to_myself)

 ─────────────────────────────────────────────────────────────────────────
                                 Time                    Allocations      
                        ───────────────────────   ────────────────────────
    Tot / % measured:        230ms /  41.0%           36.0MiB /  89.2%    

 Section        ncalls     time    %tot     avg     alloc    %tot      avg
 ─────────────────────────────────────────────────────────────────────────
 parallel_for        1   84.4ms   89.5%  84.4ms   16.6MiB   51.6%  16.6MiB
 forloop             1   9.87ms   10.5%  9.87ms   15.5MiB   48.4%  15.5MiB
 ─────────────────────────────────────────────────────────────────────────
